# Bitacora #5 - Combined_avg + Discount (Post-Paradoja)
**Fecha:** 2026-02-17  
**Autor:** Francis Bravo  

## 1. Motivacion

### Contexto de entrenamientos anteriores

**Bitácora #2 (Entrenamiento #1):**
- Parámetros: REWARD="propagate", LOSS="combined_avg", LR=5e-5, N_STATES=2
- Resultado: ❌ Q_select colapsado, Win Rate ~50% vs random

**Bitácora #3 (Entrenamiento #2):**
- Parámetros: REWARD="propagate", LOSS="combined_avg", LR=5e-4, N_STATES=6
- Resultado: ❌ Q_select colapsado, Win Rate ~45% vs random

**Bitácora #4 (Entrenamiento #1 ULTRA-AGRESIVO):**
- Parámetros: REWARD="discount", LOSS="only_select", LR=3e-4, N_STATES=8, 7 cambios simultáneos
- Resultado: ❌ Q_select colapsado, **pero descubrimiento crítico:**

### 🔴 Descubrimiento: La Paradoja del Only_Select

Al entrenar SOLO Q_select (`LOSS="only_select"`):
- Q_select (entrenada) → Colapsó a -1.0
- Q_place (NO entrenada) → **Mejoró significativamente** (aprendizaje pasivo del backbone)

**Implicación:** El entrenamiento conjunto (`combined_avg`) podría **estabilizar Q_select** aprovechando señales de Q_place.

### Hipótesis del experimento

**Si combinamos `REWARD_FUNCTION="discount"` (mejor señal temporal) con `LOSS_APPROACH="combined_avg"` (entrenamiento conjunto estabilizador), entonces Q_select NO colapsará y ambas cabezas aprenderán efectivamente.**

**Fundamento:**
1. `REWARD="discount"` demostró reducir loss 32% (bitácora 4)
2. Q_place aprendió mejor cuando el backbone se entrenaba con señales de Q_select
3. Entrenar ambas cabezas podría crear efecto estabilizador mutuo

## 2. Cambios de parametros

### 2.1 Estrategia seleccionada

**Estrategia: COMBINED_AVG + DISCOUNT (Post-Paradoja)**

Esta estrategia mantiene las mejoras de la bitácora #4 pero revierte el cambio de `LOSS_APPROACH` basándose en la Paradoja del Only_Select.

### 2.2 Cambios respecto a Bitácora #4 (Entrenamiento #1)

| Parámetro | Bitácora #4 | Bitácora #5 (NUEVO) | Razón del cambio |
|-----------|-------------|---------------------|------------------|
| **LOSS_APPROACH** | `"only_select"` | **`"combined_avg"`** | Aprovechar la Paradoja: entrenar ambas cabezas para estabilizar Q_select |

**Parámetros mantenidos de Bitácora #4:**
- ✅ REWARD_FUNCTION = "discount" (redujo loss 32%)
- ✅ LR = 3e-4 (estabilizó entrenamiento)
- ✅ GAMMA = 0.95 (necesario para discount)
- ✅ TAU = 0.01 (target network responsivo)
- ✅ N_LAST_STATES = 8 (mayor diversidad)
- ✅ TEMPERATURE_EXPLORE = 0.7 (mejor self-play)

### 2.3 Resumen de cambios vs Bitácora #2 (baseline original)

| Parámetro | Bitácora #2 (Baseline) | Bitácora #5 (NUEVO) | Diferencia |
|-----------|------------------------|---------------------|------------|
| **REWARD_FUNCTION** | `"propagate"` | **`"discount"`** | Señal temporal clara |
| **LOSS_APPROACH** | `"combined_avg"` | `"combined_avg"` | Sin cambio |
| **N_LAST_STATES** | 2/2 | **8/8** | 4x más diversidad |
| **TEMPERATURE_EXPLORE** | 2.0 | **0.7** | Mejor calidad |
| **TAU** | 0.01 | 0.01 | Sin cambio |
| **GAMMA** | 0.99 | **0.95** | Ajustado para discount |
| **LR** | 5e-5 | **3e-4** | 6x mayor |
| **BATCH_SIZE** | 30 | **64** | 2x mayor |

### 2.4 Parametros completos del entrenamiento

Parametros configurados para este experimento. Los valores **en negrita** son los que cambiaron respecto a la bitácora #2 (baseline original).

#### Configuracion principal

| Parametro | Valor | trainRL | resume | Descripcion |
|-----------|-------|---------|--------|-------------|
| STARTING_NET | `None` | [L42](trainRL.py#L42) | [L45](trainRL_resume_latest.py#L45) | Pesos aleatorios (sin checkpoint previo) |
| EXPERIMENT_NAME | `"07_COMBINED_DISCOUNT"` | [L43](trainRL.py#L43) | [L46](trainRL_resume_latest.py#L46) | Nombre del experimento |
| ARCHITECTURE | `QuartoCNN_uncoupled` | [L50](trainRL.py#L50) | [L54](trainRL_resume_latest.py#L54) | Red con cabezas independientes (mantener) |
| LOSS_APPROACH | `"combined_avg"` | [L51](trainRL.py#L51) | [L55](trainRL_resume_latest.py#L55) | Promedio de Q_place y Q_select |
| **REWARD_FUNCTION** | **`"discount"`** | [L52](trainRL.py#L52) | [L56](trainRL_resume_latest.py#L56) | Recompensas descontadas temporalmente |

#### Hiperparametros de entrenamiento

| Parametro | Valor | trainRL | resume | Descripcion |
|-----------|-------|---------|--------|-------------|
| EPOCHS | `3000` | [L66](trainRL.py#L66) | [L70](trainRL_resume_latest.py#L70) | Epocas totales |
| **BATCH_SIZE** | **`64`** | [L61](trainRL.py#L61) | [L65](trainRL_resume_latest.py#L65) | Muestras por batch (2x mayor que baseline) |
| **LR** | **`3e-4`** | [L111](trainRL.py#L111) | [L117](trainRL_resume_latest.py#L117) | Learning rate inicial (6x mayor que baseline) |
| LR_F | `1e-5` | [L112](trainRL.py#L112) | [L118](trainRL_resume_latest.py#L118) | Learning rate final |
| **TAU** | **`0.01`** | [L113](trainRL.py#L113) | [L119](trainRL_resume_latest.py#L119) | Tasa de soft update |
| **GAMMA** | **`0.95`** | [L115](trainRL.py#L115) | [L121](trainRL_resume_latest.py#L121) | Factor de descuento (ajustado para discount) |
| MAX_GRAD_NORM | `1.0` | [L110](trainRL.py#L110) | [L116](trainRL_resume_latest.py#L116) | Clipping de gradientes |
| torch.manual_seed | `5` | [L171](trainRL.py#L171) | [L204](trainRL_resume_latest.py#L204) | Semilla para reproducibilidad |

#### Generacion de experiencia

| Parametro | Valor | trainRL | resume | Descripcion |
|-----------|-------|---------|--------|-------------|
| GEN_EXPERIENCE_BY_EPOCH | `True` | [L56](trainRL.py#L56) | [L60](trainRL_resume_latest.py#L60) | Generar experiencia nueva cada epoca |
| MATCHES_PER_EPOCH | `100` | [L73](trainRL.py#L73) | [L77](trainRL_resume_latest.py#L77) | Partidas de self-play por epoca |
| **N_LAST_STATES_INIT** | **`8`** | [L69](trainRL.py#L69) | [L73](trainRL_resume_latest.py#L73) | Estados del historial (inicio, 4x vs baseline) |
| **N_LAST_STATES_FINAL** | **`8`** | [L71](trainRL.py#L71) | [L75](trainRL_resume_latest.py#L75) | Estados del historial (final) |
| **TEMPERATURE_EXPLORE** | **`0.7`** | [L91](trainRL.py#L91) | [L95](trainRL_resume_latest.py#L95) | Temperatura para exploracion (vs 2.0 baseline) |
| TEMPERATURE_EXPLOIT | `0.1` | [L94](trainRL.py#L94) | [L98](trainRL_resume_latest.py#L98) | Temperatura para explotacion |
| mode_2x2 | `True` | [L63](trainRL.py#L63) | [L67](trainRL_resume_latest.py#L67) | Victoria por cuadrado 2x2 habilitada |

#### Evaluacion y guardado

| Parametro | Valor | trainRL | resume | Descripcion |
|-----------|-------|---------|--------|-------------|
| N_MATCHES_EVAL | `30` | [L59](trainRL.py#L59) | [L63](trainRL_resume_latest.py#L63) | Partidas de evaluacion vs baselines por epoca |
| FREQ_EPOCH_SAVING | `1000` | [L96](trainRL.py#L96) | [L100](trainRL_resume_latest.py#L100) | Guardar modelo cada N epocas |
| FREQ_EPOCH_PLOT_SHOW | `50` | [L101](trainRL.py#L101) | [L105](trainRL_resume_latest.py#L105) | Mostrar plots cada N epocas |
| PLOTS_FOLDER | `...\ Mech Interp\Plots` | [L48](trainRL.py#L48) | [L52](trainRL_resume_latest.py#L52) | Carpeta de guardado de plots HTML |
| SMOOTHING_WINDOW | `10` | [L104](trainRL.py#L104) | [L110](trainRL_resume_latest.py#L110) | Ventana de suavizado para plots |
| Q_PLOT_TYPE | `"hist"` | [L107](trainRL.py#L107) | [L113](trainRL_resume_latest.py#L113) | Tipo de plot de Q-values |

## 3. Valores derivados

Valores calculados automaticamente con los parametros:

| Valor | Calculo | Bitácora #2 | Bitácora #4 | Bitácora #5 |
|-------|---------|-------------|-------------|-------------|
| STEPS_PER_EPOCH | N_LAST_STATES_FINAL * MATCHES_PER_EPOCH | 2 * 100 = **200** | 8 * 100 = **800** | 8 * 100 = **800** |
| ITER_PER_EPOCH | STEPS_PER_EPOCH // BATCH_SIZE | 200 // 30 = **6** | 800 // 64 = **12** | 800 // 64 = **12** |
| REPLAY_SIZE | 100 * STEPS_PER_EPOCH | 100 * 200 = **20,000** | 100 * 800 = **80,000** | 100 * 800 = **80,000** |
| N_BATCHS_2_UPDATE_TARGET | ITER_PER_EPOCH // 3 | 6 // 3 = **2** | 12 // 3 = **4** | 12 // 3 = **4** |

### Comparación con experimentos anteriores:

**vs Bitácora #2 (baseline):**
- **4x más experiencia** por época (800 vs 200)
- **2x más iteraciones DQN** (12 vs 6)
- **4x más replay buffer** (80k vs 20k)
- **2x más updates del target** (4 vs 2)

**vs Bitácora #4:**
- **Mismos valores derivados** (cambio solo en LOSS_APPROACH)
- Permite comparación directa del efecto de combined_avg vs only_select

## 4. Registro de entrenamientos

<a id="train-table"></a>

| # | Fecha | Script | Epocas | Parámetros clave | Resultado | Ver gráficas |
|---|-------|--------|--------|-----------------|-----------|--------------|
| [1](#train-1) | 2026-02-17 | trainRL.py | 3000 | REWARD="discount", LOSS="combined_avg", N_STATES=8, TEMP=0.7, TAU=0.01, GAMMA=0.95, LR=3e-4 | ❌ **FALLIDO** - Doble colapso: Q_place→+1.0, Q_select→-1.0 | ↓ |
| [2](#train-2) | - | - | - | - | - | ↓ |
| [3](#train-3) | - | - | - | - | - | ↓ |

---

### Resumen del entrenamiento #1

**Completado:** 2026-02-17  
**Hipótesis:** Combined_avg estabilizaría Q_select usando la señal mutua de ambas cabezas  
**Resultado:** ❌ **FALLIDO - Nuevo fenómeno descubierto: DOBLE COLAPSO**

**Métricas clave:**
- ✅ Loss: ~0.10-0.11 (mejor que bitácora #4, -35% vs ~0.16)
- ❌ Q_place: **Colapsado a +1.0** desde época ~100 (NUEVO - opuesto al de Q_select)
- ❌ Q_select: **Colapsado a -1.0** (igual que siempre)
- ➡️ Win Rate vs random: ~47-55% (sin mejora significativa)
- ➡️ Win Rate vs bot_loss-BT: ~35-40% (sin mejora)

In [ ]:
from IPython.display import display, Image, Markdown
from os import path

IMG_FOLDER = r"C:\Users\bravo\Documents\Metodos Numericos Pycharm\Mech Interp\Imagenes de la Bitacoras"

# Descomentar cuando tengas las imágenes
# display(Markdown("#### Win Rate"))
# display(Image(filename=path.join(IMG_FOLDER, "WinRate_Bitacora5_Train1.png")))

# display(Markdown("#### Q-values Progress"))
# display(Image(filename=path.join(IMG_FOLDER, "QValueProgress_Bitacora5_Train1.png")))

# display(Markdown("#### Training Loss"))
# display(Image(filename=path.join(IMG_FOLDER, "TrainingLoss_Bitacora5_Train1.png")))

# display(Markdown("#### Board Comparisons"))
# display(Image(filename=path.join(IMG_FOLDER, "BoardComparison_Bitacora5_Train1.png")))

## 5. Observaciones y resultados

### 5.1 Entrenamiento #1 - COMBINED_AVG + DISCOUNT

<a id="train-1"></a>

**Fecha:** 2026-02-17  
**Status:** ✅ COMPLETADO

#### Parámetros usados:

```python
REWARD_FUNCTION = "discount"
LOSS_APPROACH = "combined_avg"    # Cambio vs Bitácora #4 (era only_select)
ARCHITECTURE = QuartoCNN_uncoupled
LR = 3e-4
GAMMA = 0.95
TAU = 0.01
N_LAST_STATES = 8
TEMPERATURE_EXPLORE = 0.7
BATCH_SIZE = 64
```

#### Resultados finales (Época 3000):

| Métrica | Valor observado | Bitácora #4 (only_select) | Cambio |
|---------|----------------|---------------------------|--------|
| **Win Rate vs random** | ~47-55% | ~47-50% | ➡️ Sin mejora |
| **Win Rate vs bot_loss-BT** | ~35-40% | ~35-37% | ➡️ Sin mejora |
| **Q_place estado** | **Colapsado a +1.0** ⚠️ NUEVO | Diferenciación clara | ❌ Empeoró |
| **Q_place diferenciación** | **NO** | SÍ | ❌ Empeoró |
| **Q_select estado** | **Colapsado a -1.0** | Colapsado a -1.0 | ❌ Sin cambio |
| **Q_select diferenciación** | **NO** | NO | ❌ Sin cambio |
| **Loss final** | ~0.10-0.11 | ~0.16-0.17 | ✅ -35% mejor |

#### Análisis detallado por gráfica:

##### 1. Training Loss
- **Observación:** Comienza en ~0.17 y cae abruptamente a ~0.10-0.11 en las primeras ~100 épocas
- **Luego:** Se mantiene estable oscilando entre 0.09-0.12 hasta época 3000
- **Comparación:** 35% más bajo que Bitácora #4 (~0.16) y 56% más bajo que Bitácoras #2/#3 (~0.25)
- **⚠️ Interpretación crítica:** La caída abrupta inicial NO indica aprendizaje real — indica que ambas cabezas colapsaron rápidamente hacia sus extremos (+1 y -1), minimizando el MSE de forma artificial
- La oscilación posterior es ruido residual de un modelo ya colapsado

##### 2. Win Rate vs Rivals
- **vs bot_random (naranja):** Oscila entre 45-55%, media ~50%
  - Ligeramente más variable que Bitácora #4
  - Sin mejora sostenida — sigue en nivel aleatorio
- **vs bot_loss-BT (azul):** Oscila entre 30-40%, media ~37%
  - Sin cambio vs Bitácoras anteriores
- **Conclusión:** A pesar de un loss 35% menor, el gameplay no mejoró — confirma que el loss bajo es artificial (colapso), no aprendizaje real

##### 3. Q-value Progress ⚠️ **NUEVO HALLAZGO CRÍTICO: DOBLE COLAPSO**

**Q_place (fila superior) — NUEVO FENÓMENO:**
- Las 3 gráficas (R=-1, R=0, R=1) muestran **exactamente el mismo patrón**
- Línea amarilla brillante en la parte SUPERIOR (Q=+1.0) desde época ~0-100
- Todo lo demás completamente oscuro (0%)
- **Q_place colapsó a +1.0 en todas las condiciones de recompensa**
- ❌ **Sin diferenciación por recompensa** — R=-1, R=0, R=1 son idénticos
- **OPUESTO a Q_select:** Q_place satura en +1.0 mientras Q_select colapsa en -1.0

**Q_select (fila inferior) — MISMO COLAPSO QUE SIEMPRE:**
- Las 3 gráficas muestran línea amarilla en la parte INFERIOR (Q=-1.0)
- Colapsa a -1.0 desde el inicio (incluso más rápido que Bitácoras anteriores)
- ❌ Sin diferenciación por recompensa

**Patrón observado de colapso dual:**
```
Q_place → saturado en +1.0  (cabeza "optimista" — todo parece victoria)
Q_select → colapsado en -1.0 (cabeza "pesimista" — todo parece derrota)
```

##### 4. Board Comparisons
- ✅ Bug de visualización corregido — tableros muestran piezas correctamente
- Se observan estados tardíos del juego (12-15 piezas en tablero)
- Primer tablero: "27|placed|Player 1 | R=1.00" — estado ganador real
- Los tableros pre/post acción muestran diferencias mínimas entre acción del modelo y alternativas
- **Observación:** Las diferencias entre fila superior e inferior son muy pequeñas, sugiriendo que el modelo no está eligiendo acciones muy distintas a las alternativas (política casi aleatoria)

## 6. Conclusiones generales

### 6.1 Comparación con todos los entrenamientos

| Métrica | Bit. #2 | Bit. #3 | Bit. #4 (only_select) | **Bit. #5 (combined_avg)** |
|---------|---------|---------|----------------------|---------------------------|
| REWARD | propagate | propagate | discount | **discount** |
| LOSS | combined_avg | combined_avg | only_select | **combined_avg** |
| Win Rate vs random | ~0.50 | ~0.45 | ~0.47-0.50 | **~0.47-0.55** |
| Win Rate vs bot_loss-BT | ~0.37 | ~0.37 | ~0.35-0.37 | **~0.35-0.40** |
| Q_place estado | Sin diferen. | Débil | ✅ Diferenciación | ❌ **Colapsado +1.0** |
| Q_select estado | Colapsado -1.0 | Colapsado -1.0 | Colapsado -1.0 | **Colapsado -1.0** |
| Loss final | ~0.25 | ~0.25 | ~0.16 | **~0.10-0.11** |
| Loss real vs artificial | Real | Real | Real | **⚠️ ARTIFICIAL** |

### 6.2 Validación de la hipótesis

**Hipótesis:**
> "Si combinamos REWARD="discount" con LOSS="combined_avg", Q_select NO colapsará y ambas cabezas aprenderán efectivamente."

**Resultado: ❌ HIPÓTESIS REFUTADA — Resultados peores que Bitácora #4**

1. **¿LOSS="combined_avg" estabilizó Q_select?**
   - ❌ NO — Q_select colapsó igual a -1.0, incluso más rápido

2. **¿Q_place mantuvo su diferenciación?**
   - ❌ NO — Q_place, que en Bitácora #4 diferenciaba perfectamente, ahora colapsó a +1.0

3. **¿Mejoró el Win Rate?**
   - ❌ NO — Sin diferencia significativa

### 6.3 Descubrimiento principal: El Doble Colapso con Polarización Opuesta

**Hallazgo más importante:**

Con `LOSS="combined_avg"` entrenando ambas cabezas simultáneamente, la red se polarizó:

```
Q_place  → +1.0  (colapso "optimista": predice victoria en todo estado)
Q_select → -1.0  (colapso "pesimista": predice derrota en todo estado)
```

**¿Por qué ocurre esto?**

El entrenamiento conjunto con `combined_avg` crea una **dinámica adversarial implícita**:

1. Q_place recibe gradientes que la empujan hacia +1.0 (predominan las victorias de self-play donde el jugador activo suele ganar más a menudo que perder en los últimos estados)
2. Q_select recibe gradientes que la empujan hacia -1.0 (los estados de selección son más ambiguos y la señal de descuento llega más débil)
3. Al promediar los losses, la pérdida total se minimiza cuando cada cabeza va a su extremo opuesto
4. El loss artificial de ~0.10-0.11 resulta de que el MSE entre predicciones extremas (+1 y -1) y los targets descontados es bajo de forma espuria

**Implicación:** `combined_avg` NO es el mecanismo estabilizador esperado — es la causa del doble colapso.

### 6.4 Síntesis de 5 bitácoras: Mapa de colapsos

| Bitácora | LOSS | Q_place | Q_select |
|----------|------|---------|---------|
| #2 | combined_avg + propagate | Sin diferenciación | Colapsado -1.0 |
| #3 | combined_avg + propagate | Débil | Colapsado -1.0 |
| #4 | **only_select** + discount | ✅ Diferenciación CLARA | Colapsado -1.0 |
| #5 | **combined_avg** + discount | ❌ Colapsado **+1.0** | Colapsado -1.0 |

**Patrón emergente:**
- `combined_avg` → Q_place tiende a saturar en +1.0, Q_select en -1.0
- `only_select` → Q_select sigue colapsando pero Q_place mejora pasivamente
- **Q_select colapsa a -1.0 en los 4 entrenamientos sin excepción**
- **El único entrenamiento con Q_place útil fue Bitácora #4 (only_select)**

### 6.5 Confirmación: El problema es arquitectural y estructural

Después de 4 entrenamientos distintos, la evidencia es concluyente:

**Ningún cambio de hiperparámetros resuelve el colapso de Q_select:**

| Cambio intentado | Resultado en Q_select |
|-----------------|----------------------|
| LR: 5e-5 → 5e-4 → 3e-4 | Colapsado |
| GAMMA: 0.99 → 0.90 → 0.95 | Colapsado |
| TAU: 0.01 → 0.005 → 0.01 | Colapsado |
| N_LAST_STATES: 2 → 6 → 8 | Colapsado |
| REWARD: propagate → discount | Colapsado |
| LOSS: combined_avg → only_select → combined_avg | Colapsado |

**Diagnóstico final:** El colapso de Q_select es **independiente de todos los hiperparámetros probados**. Esto descarta causas de entrenamiento y confirma un **problema en la arquitectura o inicialización de la cabeza Q_select**.

### 6.6 Próximos pasos recomendados

#### Acción prioritaria: Inspección arquitectural de la cabeza Q_select 🔬

Antes de cualquier nuevo entrenamiento de hiperparámetros, se deben investigar:

**1. Análisis de gradientes:**
- Registrar la norma de los gradientes de Q_select por época
- Verificar si los gradientes son nulos (dead neurons) o explosivos desde el inicio
- Comparar con gradientes de Q_place

**2. Análisis de pesos:**
- Cargar checkpoint de época 50 (antes del colapso) y época 200 (después)
- Comparar distribución de pesos de la capa final de Q_select
- Identificar si hay neuronas con pesos saturados

**3. Probar `LOSS="only_place"`:**
- Si `only_select` mejoró Q_place, entonces `only_place` podría revelar si Q_place puede resolver el problema sola
- Permite aislar completamente la dinámica de Q_place sin interferencia de Q_select

**4. Ajustar activación final de Q_select:**
- Si usa `tanh` → Probar sin activación (Q-values sin límite)
- Si usa `ReLU` → Dead ReLU posible, probar `LeakyReLU`
- Verificar que la función de activación sea compatible con Q-values negativos

### 6.7 Conclusión final

**Estado del proyecto tras 5 bitácoras:**
- ❌ Q_select colapsa de forma estructural e inevitable
- ❌ El win rate no supera el nivel aleatorio en ningún entrenamiento
- ✅ Identificamos que `only_select` da mejores resultados que `combined_avg` para Q_place
- ✅ Mapeamos el comportamiento de colapso en múltiples condiciones
- ✅ El loss bajo de Bitácora #5 es artificial (colapso dual), no aprendizaje real

**Lección crítica:**
> Un loss bajo NO garantiza aprendizaje. En este caso, el loss de ~0.10 es consecuencia del colapso dual de ambas cabezas a sus extremos, no de haber aprendido la política óptima.

**Diagnóstico definitivo:**
> El sistema no puede aprender con la arquitectura y señales actuales. La siguiente acción debe ser **investigación arquitectural**, no más cambios de hiperparámetros.